## Installation

In [1]:
%%capture
!pip install transformers==4.38.0 sentence-transformers
!pip install --upgrade  langchain langchain-community langchainhub langchain-openai chromadb
!pip install -qU langchain-openai
!pip install evaluate
!pip install rouge_score

## Authorization

- you need an Hugging Face account
- get your token from here https://huggingface.co/settings/tokens

In [2]:
import getpass, os

os.environ["HF_API_TOKEN"] = "YOUR_TOKEN_HERE"
os.environ["OPENAI_API_KEY"] = "sk-YOUR_TOKEN_HERE"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "YOUR_TOKEN_HERE"

## Imports

In [3]:
import pandas as pd
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.runnables import RunnableParallel
from tqdm.auto import tqdm

tqdm.pandas()

## Read data

In [5]:
# Export the DataFrame to a CSV file
csv_file_path = 'data_alzheimer - Sheet1.csv'

loader = CSVLoader(file_path=csv_file_path, source_column="reponse", metadata_columns=["question"])
data = loader.load()

In [6]:
data[0]

Document(page_content='reponse: Alzheimer disease is a progressive neurological disorder that affects memory thinking and behavior leading to cognitive decline and impairments in daily functioning', metadata={'source': 'Alzheimer disease is a progressive neurological disorder that affects memory thinking and behavior leading to cognitive decline and impairments in daily functioning', 'row': 0, 'question': "What is Alzheimer's disease, and how does it impact memory, cognition, and behavior, ultimately resulting in a decline in daily functioning?"})

## Retrieval Augmented Generation (RAG) Pipeline

In [7]:
# Initialize a text splitter with specific chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# Split the input data into manageable chunks
splits = text_splitter.split_documents(data)
# Create a vector store from the document splits using OpenAI embeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Convert the vector store into a retriever for fetching relevant documents
retriever = vectorstore.as_retriever()
# Pull a specific prompt model from a hub
prompt = hub.pull("rlm/rag-prompt")
# Initialize a language model for generating responses
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    """
    Function to format documents into a single string
    """
    return "\n\n".join(doc.page_content for doc in docs)

# Define a chain for processing documents through the prompt and language model
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# Set up a parallel processing chain for handling context retrieval and question answering
rag_chain = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

## Demo

In [8]:
res = rag_chain.invoke("What is alzheimer?")
print(res)
print()
res['answer']

{'context': [Document(page_content='reponse: Alzheimer disease is a progressive neurological disorder that affects memory thinking and behavior leading to cognitive decline and impairments in daily functioning', metadata={'question': "What is Alzheimer's disease, and how does it impact memory, cognition, and behavior, ultimately resulting in a decline in daily functioning?", 'row': 0, 'source': 'Alzheimer disease is a progressive neurological disorder that affects memory thinking and behavior leading to cognitive decline and impairments in daily functioning'}), Document(page_content="reponse: Alzheimer's disease is a brain disorder that gets worse over time", metadata={'question': "How does Alzheimer's disease progress over time?", 'row': 19, 'source': "Alzheimer's disease is a brain disorder that gets worse over time"}), Document(page_content='reponse: Alzheimer’s disease is the most common type of dementia.', metadata={'question': 'What is the most prevalent form of dementia?', 'row'

"Alzheimer's disease is a progressive neurological disorder that affects memory, thinking, and behavior, leading to cognitive decline and impairments in daily functioning. It is the most common type of dementia and the most common cause of a gradual decline in memory, thinking, behavior, and social skills."

In [9]:
res = rag_chain.invoke("What is alzheimer symptoms?")
print(res)
print()
res['answer']

{'context': [Document(page_content='reponse: symptoms of alzheimer included memory loss, language problems, and unpredictable behavior', metadata={'question': "What are some common symptoms of Alzheimer's disease, encompassing memory loss, language difficulties, and unpredictable behavioral changes?", 'row': 10, 'source': 'symptoms of alzheimer included memory loss, language problems, and unpredictable behavior'}), Document(page_content='reponse: Symptoms of Alzheimer disease include memory loss difficulty solving problems confusion about time or place changes in mood or personality and challenges with communication and language', metadata={'question': "What are the symptoms of Alzheimer's disease, encompassing memory loss, problem-solving difficulties, confusion, mood changes, personality alterations, and communication challenges?", 'row': 1, 'source': 'Symptoms of Alzheimer disease include memory loss difficulty solving problems confusion about time or place changes in mood or person

"Alzheimer's symptoms include memory loss, language problems, and unpredictable behavior. Symptoms can also include difficulty solving problems, confusion about time or place, changes in mood or personality, and challenges with communication and language. As the disease progresses, symptoms may worsen, leading to disorientation, mood and behavior changes, and deepening confusion about events, time, and place."

## Evaluation

In [10]:
df = pd.read_csv("data_alzheimer - Sheet1.csv")
df["LLM_response"] = df["question"].progress_apply(rag_chain.invoke)

  0%|          | 0/95 [00:00<?, ?it/s]

In [11]:
import evaluate

rouge = evaluate.load("rouge")

In [15]:
df["LLM_answer"] = df["LLM_response"].apply(lambda x: x["answer"])

In [82]:
import numpy as np

BASE_EMBEDDINGS = OpenAIEmbeddings()

def cosine_similarity(vector1, vector2):
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    similarity = dot_product / (norm_vector1 * norm_vector2)
    return similarity

def cosine_sim(sentence_1, sentence_2, embeddings=BASE_EMBEDDINGS):
    emb_1 = np.array(embeddings.embed_documents(sentence_1))
    emb_2 = np.array(embeddings.embed_documents(sentence_2))
    scores = [cosine_similarity(em1, em2) for em1, em2 in zip(emb_1, emb_2)]
    return scores

In [84]:
df["score"] = df.progress_apply(lambda row: rouge.compute(predictions=[row["LLM_answer"]], references=[row["reponse"]], rouge_types=['rouge1'])["rouge1"], axis=1)

df["cosine_sim"] = cosine_sim(df["LLM_answer"].values, df["reponse"].values)

  0%|          | 0/95 [00:00<?, ?it/s]

In [88]:
print(f"Correlation Score between generated predictions and answers: {df['cosine_sim'].mean():.4f}")

Correlation Score between generated predictions and answers: 0.9344


In [16]:
rouge_score = rouge.compute(predictions=df["LLM_answer"].tolist(), references=df["reponse"].tolist(), rouge_types=['rouge1'])["rouge1"]
print(f"Rouge Score on all data: {rouge_score}")

Rouge Score on all data: 0.4585430035632728


The differences in scores are explained by the fact that the rouge score is a hard metric which tells us how much overlap between the target and the prediction, whereas, the cosine similarity compare the semantic meaning.